In [1]:
import pandas as pd
import numpy as np

In [2]:
exp_name = "exp_compass"
exp_number = "exp_1"
base_path = "/Users/andreasathanasopoulos/Phd/projects/bayesian_fairness/"
data_path = base_path + "/my_code/Bayesian-fairness/data"
save_path = base_path + f"/my_code/Bayesian-fairness/results/{exp_name}/{exp_number}"

# load data


In [3]:
# set atributes
Z_atr = ["sex", "race"]
X_atr = ['age_cat', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree']
Y_atr = ['two_year_recid']

In [4]:
def load_compas_dataset(path, clip_features, clip_value):
    raw_data = pd.read_csv(path + "/compas.csv")
    raw_data[raw_data[clip_features] > 2] = 2
    return raw_data

In [5]:
clip_features = ["juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count"]
data = load_compas_dataset(path = data_path, clip_features = clip_features, clip_value = 2)

In [6]:
# get distinct values
unique_z = np.unique(data[Z_atr].values, axis=0)
n_z = len(unique_z)

unique_x = np.unique(data[X_atr].values, axis=0)
n_x = len(unique_x)

unique_y = np.unique(data[Y_atr].values, axis=0)
n_y = len(unique_y)

In [7]:
print("Unique Z values:", n_z)
print("Unique X values:", n_x)
print("Unique Y values:", n_y)

Unique Z values: 12
Unique X values: 141
Unique Y values: 2


In [8]:
def encode_data(data,unique_values):
    encoded_value = np.array([])
    for i, d in data.iterrows():
        # encode feature to an index represents the unique value.
        index = np.argmax((d.values == unique_values).all(axis=1))
        encoded_value = np.append(encoded_value, index)
    return encoded_value.astype(int)

In [9]:
def create_compact_dataset():
    # encode z
    encoded_z = encode_data(data[Z_atr], unique_z)
    assert (int(max(encoded_z))+1) == len(unique_z)
    
    # encode x
    encoded_x = encode_data(data[X_atr], unique_x)
    assert (int(max(encoded_x))+1) == len(unique_x)
    
    # encode y
    encoded_y = encode_data(data[Y_atr], unique_y)
    assert (int(max(encoded_y))+1) == len(unique_y)

    return np.stack([encoded_x, encoded_z, encoded_y],axis=1)

In [10]:
dataset = create_compact_dataset()
dataset = pd.DataFrame(dataset,columns = ["x","z", "y"])

In [11]:
dataset.head()

,x,z,y
0,124,0,0
1,64,1,1
2,11,1,1
3,20,1,0
4,68,0,0


In [12]:
train_data = dataset.iloc[0:6000]
test_data = dataset.iloc[6000:]

In [13]:
print("training size:", train_data.shape)
print("testing size:", test_data.shape)

training size: (6000, 3)
testing size: (1214, 3)


# model

In [14]:
class DirichletModel:
    def __init__(self, n_x, n_z, n_y, prior):
        self.X = n_x
        self.Y = n_y
        self.Z = n_z
        
        # why we have this fractoriazation?
        self.N_x = prior + np.zeros(shape = (n_x, 1) )
        self.N_y_x = prior + np.zeros(shape = (n_y, n_x) )
        self.N_z_yx = prior + np.zeros(shape = (n_z, n_y, n_x) )
        
        # prop
        self.Px = np.zeros(shape = (n_x, 1) )
        self.Py_x = np.zeros(shape = (n_y, n_x) )
        self.Pz_yx = np.zeros(shape = (n_z, n_y, n_x) )
    
    def update_posterior_belief(self, data):
        for i , datum in data.iterrows():
            self.N_x[datum["x"]] += 1
            self.N_y_x[datum["y"], datum["x"]] += 1
            self.N_z_yx[datum["z"], datum["y"], datum["x"]] += 1
        
    def calculate_marginal_propabilities(self):
        # calculate Prop
        self.Pxyz =  np.zeros(shape = (self.X, self.Y, self.Z) )
        self.Pxy = np.zeros(shape = (self.X, self.Y) )
        self.Pyz = np.zeros(shape = (self.Y, self.Z) )
        self.Px_yz = np.zeros(shape = (self.X, self.Y, self.Z) )
        
        # calculate Pxyz
        for x in range(self.X):
            for y in range(self.Y):
                for z in range(self.Z):
                    # calculate Pxyz
                    self.Pxyz[x, y, z] = self.Pz_yx[z, y, x] * self.Py_x[y, x] * self.Px[x]
        
        # calculate Px_yz 
        for y in range(self.Y):
            for z in range(self.Z):
                self.Pyz[y, z] = np.sum(self.Pxyz[:, y, z])    
                for x in range(self.X):
                    self.Pxy[x, y] = np.sum(self.Pxyz[x, y, :])
                    self.Px_yz[x, y, z] = self.Pxyz[x, y, z] / self.Pyz[y, z]
        
        # calculate Px_y
        self.Py = np.zeros(shape = (self.Y, 1) )
        self.Px_y = np.zeros(shape = (self.X, self.Y) )
        for y in range(self.Y):
            self.Py[y] = np.sum(self.Pxyz[:, y, :])
            for x in range(self.X):
                self.Px_y[x,y] = self.Pxy[x,y] / self.Py[y]
        
        # calculate Pz_y
        self.Pz_y = np.zeros(shape = (self.Z, self.Y) )
        for y in range(self.Y):
            self.Pz_y[:,y] = self.Pyz[y,:] / self.Py[y]
                
    def get_marginal_model(self):
        self.Px = self.N_x / np.sum(self.N_x)
        self.Py_x = self.N_y_x / np.sum(self.N_y_x, axis=0)
        self.Pz_yx = self.N_z_yx / np.sum(self.N_z_yx, axis=0)
        self.calculate_marginal_propabilities()
        
    def sample_model(self):
        self.Px = np.random.dirichlet( np.ravel(dirichlet_model.N_x) ).reshape((-1,1))
        
        for y in range(self.Y):
            self.Py_x[y, :] = np.random.dirichlet(self.N_y_x[y,:])
        
        for y in range(self.Y):
            for z in range(self.Z):
                self.Pz_yx[z, y, :] = np.random.dirichlet(self.N_z_yx[z, y, :])
        
        self.calculate_marginal_propabilities()

In [15]:
dirichlet_prior = 0.5
true_dirichlet_model = DirichletModel(n_x = n_x, n_z = n_z, n_y = n_y, prior = dirichlet_prior)

In [16]:
true_dirichlet_model.update_posterior_belief(data=test_data)
true_dirichlet_model.get_marginal_model()

# Functions

#### policy

In [17]:
def get_random_policy(size):
    a = np.ones(shape=size[0])
    policy = np.random.dirichlet(a, size= size[1])
    return np.transpose(policy)

def get_random_policy_2(size):
    policy = np.random.random(size = size)
    return policy

In [18]:
policy = get_random_policy(size = (2,2))

In [19]:
def normilize_policy(policy):
    policy[policy < 0] = 0
    policy[policy > 1] = 1
    for x in range(policy.shape[-1]):
        policy[:, x] /= np.sum(policy[:, x])
    return policy

#### delta

In [20]:
def get_delta(Px_y, Px_yz):
    delta = np.zeros(Px_yz.shape)
    for z in range(Px_yz.shape[-1]):
        delta[:, :, z] = Px_y - Px_yz[:,:,z]
    return delta

def get_delta(Px_y, Px_yz, Pz_y):
    delta = np.zeros(Px_yz.shape)
    for z in range(Px_yz.shape[-1]):
        delta[:, :, z] = (Px_y - Px_yz[:,:,z]) * Pz_y[z,:]
    return delta

In [21]:
# d_1 = get_delta(true_dirichlet_model.Px_y, true_dirichlet_model.Px_yz)

In [22]:
# d_1

In [23]:
# d_2 = get_delta2(true_dirichlet_model.Px_y, true_dirichlet_model.Px_yz, true_dirichlet_model.Pz_y)

In [24]:
# d_2

#### utility

In [25]:
def get_eye_utility(size):
    return np.eye(size)

# Algorithm

### 1. set parameters

In [26]:
# --- data parameters
horizon = train_data.shape[0]

num_X = n_x # number of features
num_Y = n_y # number of outcomes
num_Z = n_z # number of sensitive features
num_A = 2 # number of actions

# --- SGD parameters
n_iter = 400 # number of itteration for SGD
lr = 1.0 # learning rate

# --- Algorithm parameters
update_policy_period = 100 # period to update policy
l = 0.5 # lambda
n_samples = 16 # number of sample for bayssian policy

# --- Utility
utility = get_eye_utility(size=num_A)

### 2. initializition

In [27]:
# initialize belief
belief = DirichletModel(n_x = num_X, n_y=num_Y, n_z=num_Z, prior = 0.5)

# initialize policy
policy = get_random_policy(size = (num_A, num_X))

In [28]:
# get true model delta to avoid computations
true_model_delta = get_delta(true_dirichlet_model.Px_y,
                             true_dirichlet_model.Px_yz,
                             true_dirichlet_model.Pz_y)

### 3.  main loop

#### fairness functions

In [29]:
def get_fairness(policy, model_delta):
    (X, Y, Z) = model_delta.shape
    fairness = 0
    for y in range(Y):
        for z in range(Z):
            delta = np.matmul(policy, model_delta[:, y , z ])
            fairness += np.linalg.norm(delta, 1)
    return fairness

In [30]:
def get_fairness_gradient(policy, model_delta):
    """
    Todo: vectorize operation
    """
    fairness_gradient = np.zeros(policy.shape)
    
    (X, Y, Z) = model_delta.shape
    for y in range(Y):
        for z in range(Z):
            dyz = model_delta[:,y,z].reshape((-1,1))
            c = np.matmul(policy, dyz)
            fairness_gradient -= np.matmul(c, dyz.T)
    
    return fairness_gradient

#### utility functions

In [31]:
def get_utility(policy, model, utility):
    """
    Calculate expected utility
    Todo: vectorize operation - minor
    """
    A, X = policy.shape
    Y = A
    Eu = 0
    for x in range(X):
        for y in range(Y):
            for a in range(A):
                Eu += utility[a,y] * policy[a,x] * model.Pxy[x,y]
                
    return Eu

In [32]:
def get_utility_gradient(policy, model, utility):
    """
    Todo: vectorize operation
    """
    utility_gradient = np.matmul(utility, model.Pxy.T )
    
    
    return utility_gradient

#### gradient fun

In [33]:
def project_gradient(grad):
    proj_grad = grad - np.mean(grad,axis=0)
    return proj_grad

#### opt functions

In [34]:
def evaluate(true_model, true_model_delta, policy, utility, l):
    """
    Evaluate policy on true model
    """
    results = {}
    results["fairness"] = np.round(get_fairness(policy, true_model_delta),4)
    results["utility"] = get_utility(policy, true_model, utility)
    results["total"] = (1 - l) * results["utility"] - l * results["fairness"]
    return results

In [35]:
def update_policy(policy, model, utility, l, lr, n_iter):
    """
    Marginal Policy Dirichlet
    """
    model.get_marginal_model()
    model_delta = get_delta(model.Px_y, model.Px_yz, model.Pz_y)
    
    for i in range(n_iter):
        fairness_gradient = get_fairness_gradient(policy, model_delta)
        utility_gradient = get_utility_gradient(policy, model, utility)
        gradient = (1 - l) * utility_gradient + l * fairness_gradient # minus on the gradient calc.
        gradient = project_gradient(gradient)
        policy = policy + lr * gradient # maximize Utility & minimize fairness constrain.
        policy = normilize_policy(policy)
    
    return policy

In [37]:
# reproduce results
stop

NameError: name 'stop' is not defined

In [38]:
def load_org_policy(org_path_results):
    file = pd.read_csv(org_path_results + "/policy.csv")
    p_1 = file.iloc[4].values
    p_2 = file.iloc[5].values
    p_1 = [float(x) for x in p_1[0].split(" ")[1:]]
    p_2 = [float(x) for x in p_2[0].split(" ")[1:]]
    return np.array([p_1, p_2])
# [float(x) for x in p_1[0].split(" ")[1:]]

In [39]:
org_path_results = "/Users/andreasathanasopoulos/Phd/projects/bayesian_fairness/org_code/bayesian-fairness/src/octave"
policy = load_org_policy(org_path_results)
policy.shape

(2, 141)

In [40]:
l = 0.9

In [ ]:
def get_utility(policy, model, utility):
    """
    Calculate expected utility
    Todo: vectorize operation - minor
    """
    A, X = policy.shape
    Y = A
    Eu = 0
    for x in range(X):
        for y in range(Y):
            for a in range(A):
                Eu += utility[a,y] * policy[a,x] * model.Pxy[x,y]
                
    return Eu

In [60]:
get_utility(policy, true_dirichlet_model, utility)

0.637832923125326

In [61]:
np.sum(np.mean(m,axis=0))

0.5000000000000001

In [62]:
m = np.matmul(policy,true_dirichlet_model.Pxy)

In [63]:
np.sum(utility * m)

0.6378329231253261

In [64]:
import tensorflow as tf

In [66]:
tf.one_hot(range(141), 1)

<tf.Tensor: shape=(141, 1), dtype=float32, numpy=
array([[1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
 

In [59]:
steps = horizon // update_policy_period

results = []
for step in range(steps):    
    # update policy step
    policy = update_policy(policy, belief, utility, l, lr, n_iter) # SDG to update policy
    
    # evaluation step
    step_results = evaluate(true_dirichlet_model, true_model_delta, policy, utility, l)
    results += [step_results]
    
    # update belief step
    data_start_index = step * update_policy_period
    data_stop_index = min(data_start_index + update_policy_period, horizon)
    belief.update_posterior_belief(train_data.iloc[data_start_index : data_stop_index])
    
    print(f"--- Step : {data_start_index + 1} \n  ------- {step_results}")
    

--- Step : 1 
  ------- {'fairness': 0.1384, 'utility': 0.4804503987963803, 'total': -0.07651496012036199}
--- Step : 101 
  ------- {'fairness': 0.1845, 'utility': 0.5518809227813718, 'total': -0.11086190772186283}
--- Step : 201 
  ------- {'fairness': 0.2297, 'utility': 0.5796560575888795, 'total': -0.14876439424111207}
--- Step : 301 
  ------- {'fairness': 0.2704, 'utility': 0.608810747225974, 'total': -0.18247892527740261}
--- Step : 401 
  ------- {'fairness': 0.263, 'utility': 0.6187054138137963, 'total': -0.1748294586186204}
--- Step : 501 
  ------- {'fairness': 0.2485, 'utility': 0.6228421039832859, 'total': -0.16136578960167144}
--- Step : 601 
  ------- {'fairness': 0.2559, 'utility': 0.6280360529013993, 'total': -0.16750639470986012}
--- Step : 701 
  ------- {'fairness': 0.2679, 'utility': 0.6339927125354835, 'total': -0.17771072874645166}
--- Step : 801 
  ------- {'fairness': 0.2676, 'utility': 0.63539189184272, 'total': -0.17730081081572802}
--- Step : 901 
  ------- 

In [57]:
np.log(0.6)

-0.5108256237659907

In [58]:
np.log(0.2)

-1.6094379124341003

In [41]:
evaluate(true_dirichlet_model, d_2, policy, utility, l)

NameError: name 'd_2' is not defined

In [ ]:
train_data.iloc[data_start_index : data_stop_index]

In [ ]:
pd_resutls = pd.DataFrame(results)

# plots

In [ ]:
pd_resutls[["utility"]].plot()

In [ ]:
# load org resutls
org = np.loadtxt(org_path_results + "/results.csv")
org = org[:,1][1:].reshape((4,-1))

org_pd = pd.DataFrame( columns= ["utility","fairness","total"])
org_pd["utility"] = org[0]
org_pd["fairness"] = org[2]
org_pd["total"] = org[1]

In [ ]:
org_pd["utility"].plot()
pd_resutls["utility"].plot()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(org_pd["utility"], label = "original code results")
plt.plot(pd_resutls["utility"], label = "new code results")
plt.title("Utility")
plt.legend()
plt.show()

plt.figure()
plt.plot(org_pd["fairness"], label = "original code results")
plt.plot(pd_resutls["fairness"], label = "new code results")
plt.title("Fairness")
plt.legend()
plt.show()


In [ ]:
org_pd["fairness"].plot()
pd_resutls["fairness"].plot()

# Questions

In [ ]:
1. ProjectPolicyGradient ???